# ROS Basic - part 3


## Creating My First Package

I start off from [here.](https://index.ros.org/doc/ros2/Tutorials/Creating-Your-First-ROS2-Package/#createpkg)

so I guess, I shall need some of the stuff from previous example to start with.


I start with a dockerfile in the my_pkg directory. I added some of the parts I thought would be needed.

okay then we will have some 'required' files for the python package

```
package.xml file containing meta information about the package

setup.py containing instructions for how to install the package

setup.cfg is required when a package has executables, so ros2 run can find them

/<package_name> - a directory with the same name as your package, used by ROS 2 tools to find your package, contains __init__.py

```


I create and put these in my_ws for now.


so for the structure of the package for python I see two variants

```
package_name/
    setup.py
    package.xml
    package_name/
```
and

```
pacakge_name/
    setup.py
    package.xml
    resource/package_name/
```

I am going with the second one for now. Also need to change the name my_ws







looks like we use this to initialize the pckage:

```
ros2 pkg create --build-type ament_python <package_name>
```

>For this tutorial, you will use the optional argument --node-name which creates a simple Hello World type executable in the package.

```
ros2 pkg create --build-type ament_python --node-name my_node my_package
```

okay again running the above command in docker file returns error, but it runs okay in the container.?!


hmm, looking at what the ros actually created, I see this structure:

```
my_first_package  package.xml  resource  setup.cfg  setup.py  test
```

and
```
root@d7f6ec0f8f42:/opt/ros/overlay_ws/src/my_first_package/my_first_package# ls
__init__.py  my_node.py
```

and

```
root@d7f6ec0f8f42:/opt/ros/overlay_ws/src/my_first_package# ls test
test_copyright.py  test_flake8.py  test_pep257.py
```

and

```
root@d7f6ec0f8f42:/opt/ros/overlay_ws/src/my_first_package# ls resource/
my_first_package
```

The interesting part is that this last part seems to be only an empty file

Aha! I guess I found out.

first it looked to me that the reason that the ros2 is not recognized is that in the container 
we need to source it (or is it done by default when we call the image?)
so then the question was how to source the ros?

looking at [this](https://stackoverflow.com/questions/20635472/using-the-run-instruction-in-a-dockerfile-with-source-does-not-work) 
and [this](https://stackoverflow.com/questions/4732200/replacement-for-source-in-sh)

[this also seems related](https://stackoverflow.com/questions/60545462/build-a-docker-image-ros2-when-try-tu-run-install-setup-bash-in-dockerfile-n)


I tried a few combos:

```
RUN /bin/bash -c "source /opt/ros/foxy/setup.bash"

RUN ros2 pkg create --build-type ament_python --node-name my_node my_first_package
```

which got me this error:

```
#8 0.236 /bin/sh: 1: ros2: not found
```

okay thought maybe instead of bin/bash I should try bin/sh

```
RUN /bin/sh -c "source /opt/ros/foxy/setup.bash"

RUN ros2 pkg create --build-type ament_python --node-name my_node my_first_package
```

which got me this error:

```
 => ERROR [4/5] RUN /bin/sh -c "source /opt/ros/foxy/setup.bash"                                                                                     0.3s
```

okay though that I am calling a bash file with sh, so I tried:

```
 RUN /bin/sh -c "source /opt/ros/foxy/setup.sh"

RUN ros2 pkg create --build-type ament_python --node-name my_node my_first_package
```

which got me this error:

```
=> ERROR [4/5] RUN /bin/sh -c "source /opt/ros/foxy/setup.sh"                                                                                       0.3s
------
 > [4/5] RUN /bin/sh -c "source /opt/ros/foxy/setup.sh":
#7 0.237 /bin/sh: 1: source: not found
```

thought this is not working. so checking back to the QA thread, I saw that we can just run the bash file

```
RUN  . /opt/ros/foxy/setup.sh 

RUN ros2 pkg create --build-type ament_python --node-name my_node my_first_package

```

which completes the first one but the second one fails. Then I thought maybe these two commands are executed independently 
in the build mode. So I combined them and it works fine.

```
RUN  . /opt/ros/foxy/setup.sh && \

 ros2 pkg create --build-type ament_python --node-name my_node my_first_package

```


So in order to build and run the package the followings are suggested:
```
colcon build
## or colcon build --packages-select my_package
. install/setup.bash

ros2 run my_package my_node

```

I coppied the same stuff again into the docker.

```
RUN colcon build
RUN . install/setup.bash
```

but again when going into container I had to source it again

```
source install/setup.bash
```

so Q here is why the last step is not recognized.
it is a source type command. for the Ros2 the source was not needed.

well looking at the setup.bash it seems related to extending the environment.
hmm still not clear!

I guess for these questions I need to do some better search ( or ask Alex!)

But before that, it would be good to examine the codes generated in python for the node example.

## Writing a simple publisher and subscriber (Python)

from [here](https://index.ros.org/doc/ros2/Tutorials/Writing-A-Simple-Py-Publisher-And-Subscriber/#pypubsub)